In [1]:
import json
import torch

def read_data(readPath):

    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)

    return taskData

In [2]:
import os
import sys

sys.path.append('../')

from models.model import multiTaskModel

def load_params(model_file):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Load finetuned model 
    loadedDict = torch.load(model_file, map_location=torch.device(device))
    
    taskParams = loadedDict['task_params']
    allParams = {}
    allParams['task_params'] = taskParams
    allParams['gpu'] = torch.cuda.is_available()
    
    # dummy values
    allParams['num_train_steps'] = 10
    allParams['warmup_steps'] = 0
    allParams['learning_rate'] = 2e-5
    allParams['epsilon'] = 1e-8

   
    return allParams, loadedDict

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-09 00:07:43.234887: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 00:07:43.289331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 00:07:44.182453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

In [4]:
import os
from model.custom_dataset import CustomDataset
def get_embedding_finetuned(dataDir, readFile, model, wriDir):
    data = read_data(os.path.join(dataDir, readFile))
    line = data[0]

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Convert inputs to PyTorch tensors
    attention_mask = torch.tensor([int(x) for x in line['attention_mask']], device=device)
    tokens_tensor = torch.tensor([int(x) for x in line['token_id']],  device=device)
    segments_tensors = torch.tensor([int(x) for x in line['token_type_ids']],  device=device)
    
    test_dataset = CustomDataset(
        data_path=dataDir,
        file_name=readFile)
    
    print(test_dataset)
    print(type(test_dataset))
    # with torch.no_grad():
    #     outputs_model, _ = model.network(test_dataset['atte'], segments_tensors, attention_mask, 0, 'conllsrl')
       
    #     last_hidden_states = outputs_model[0]
    #     hidden_states = outputs_model[2]
    
    # # sum of last four layer
    # word_embed_5 = torch.stack(hidden_states[-4:]).sum(0)
    
    # # concatenate last four layers
    # word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)
    
    # return last_hidden_states, word_embed_5, word_embed_6


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-04-10 16:12:30.952074: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification m

In [3]:
model_file = '../output/multi_task_model_9_13050.pt'
allParams, loadedDict = load_params(model_file)
model = multiTaskModel(allParams)
model.load_multi_task_model(loadedDict)

dataDir = './mlm_prepared_data_3/'
readFile = 'dev_mlm.json'
wriDir = 'test_get_embedding_MLM_data'
# last_hidden_states_finetuned, sum_4_finetuned, concat_4_finetuned = get_embedding_finetuned(dataDir, readFile, model, wriDir)

In [6]:
loadedDict['task_params'].labelMap['conllsrl']


{'B-A1': 0,
 'I-A1': 1,
 'O': 2,
 'B-V': 3,
 'B-A0': 4,
 'I-A0': 5,
 'B-A4': 6,
 'I-A4': 7,
 'I-A3': 8,
 'B-A2': 9,
 'I-A2': 10,
 'B-A3': 11,
 '[CLS]': 12,
 '[SEP]': 13,
 'X': 14}

: 

In [ ]:
from mlm_finetune import load_model
import torch
model_file = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/output/multi_task_model_9_13050.pt'     
device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

min_val_loss = load_model(model_file, model, device, optimizer, scheduler)